### Домашнее задание по обработке текстов
Предсказание цены акции по экономическим новостям

Входные данные:

    1.Новости о компании "Газпром", начиная с 2010 года
- преообразуем колонку дата в формат даты.    


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('texts.csv')
df.date =  pd.to_datetime(df.date, format='%d.%m.%Y')
df['len'] = df.text.apply(len)
df.set_index('date', inplace=True)
df.head()

,text,len
date,,
2017-11-09,Компания рассчитывает на решение по газовому с...,419
2017-11-08,"Как и предполагал “Ъ”, «Газпром», воспользова...",624
2017-11-01,Новая редакция американских санкций ставит по...,835
2017-10-30,"Как стало известно “Ъ”, известный на рынке ри...",436
2017-10-23,"НОВАТЭК, который через пять лет собирается за...",1272


    2.Стоимость акций компании "Газпром" на ММВБ, начиная с 2010 года
        цена открытия (Open)
        цена закрытия (ClosingPrice)
        максимальная цена за день (DailyHigh)
        минимальная цена за день (DailyLow)
        объем бумаг (VolumePcs)
        
- добавим колонку "Дельта" - разница между открытием и закрытием, преобразуем колонки в нужные форматы "Дата" или "Число"

In [2]:
pr_all = pd.read_csv('gazprom_prices.csv', sep=';')
pr_all['date'] =  pd.to_datetime(pr_all.Date, format='%d.%m.%Y')
pr_all.drop(['Date'], axis=1, inplace=True)
pr_all['Open'] = pr_all['Open'].map(
            lambda pr_all: np.float32(pr_all.replace(',', '.')),na_action='ignore')
pr_all['ClosingPrice'] = pr_all['ClosingPrice'].map(
            lambda pr_all: np.float32(pr_all.replace(',', '.')),na_action='ignore')
pr_all['DailyHigh'] = pr_all['DailyHigh'].map(
            lambda pr_all: np.float32(pr_all.replace(',', '.')),na_action='ignore')
pr_all['DailyLow'] = pr_all['DailyLow'].map(
            lambda pr_all: np.float32(pr_all.replace(',', '.')),na_action='ignore')
pr_all['delta'] = pr_all.Open - pr_all.ClosingPrice

pr_all.set_index('date', inplace=True)
pr_all.head(5)

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta
date,,,,,,
2017-12-08,133.429993,132.600006,133.899994,132.000000,16037970,0.829987
2017-12-07,133.699997,133.020004,133.869995,132.809998,18198430,0.679993
2017-12-06,133.330002,134.000000,134.289993,132.910004,14641730,-0.669998
2017-12-05,133.479996,133.649994,133.990005,132.779999,12684800,-0.169998
2017-12-04,133.009995,133.770004,134.000000,131.929993,17818980,-0.760010


#### Часть 1. Вводная

Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. 

Собираем общую таблицу:

In [3]:
Data = pr_all.join(df)
Data.dropna(axis=0, inplace=True)
Data.head()

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,text,len
date,,,,,,,,
2017-11-09,132.309998,131.500000,132.820007,131.139999,33869650,0.809998,Компания рассчитывает на решение по газовому с...,419.0
2017-11-08,132.000000,132.300003,133.940002,131.580002,39381960,-0.300003,"Как и предполагал “Ъ”, «Газпром», воспользова...",624.0
2017-11-01,126.400002,126.500000,126.889999,125.970001,18232550,-0.099998,Новая редакция американских санкций ставит по...,835.0
2017-10-30,125.959999,125.980003,126.930000,125.529999,19263340,-0.020004,"Как стало известно “Ъ”, известный на рынке ри...",436.0
2017-10-23,127.050003,126.800003,127.470001,126.370003,17308800,0.250000,"НОВАТЭК, который через пять лет собирается за...",1272.0


In [4]:
Data.describe()

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,len
count,1145.000000,1145.000000,1145.000000,1145.000000,1.145000e+03,1145.000000,1145.000000
mean,150.018681,149.902970,151.635738,148.193511,4.492943e+07,0.115712,772.908297
std,23.464697,23.390248,23.702841,23.090757,2.455151e+07,2.316764,546.978587
min,107.230003,107.169998,109.199997,106.500000,7.642310e+06,-12.639999,85.000000
25%,135.889999,135.770004,137.000000,134.500000,2.859430e+07,-1.120003,406.000000
50%,144.270004,144.300003,145.889999,142.669998,3.843521e+07,0.110001,584.000000
75%,157.169998,156.690002,158.630005,155.149994,5.580905e+07,1.310013,1010.000000
max,243.520004,244.000000,247.470001,241.770004,2.024663e+08,13.849998,4120.000000


In [5]:
%%time 
from pymystem3 import Mystem

def lemma(text):
    text = text.replace('\xa0', ' ')
    try:
        return "".join(Mystem().lemmatize(text)).strip()  
    except:
        return " "

Data['lem_text'] = Data.text.apply(lemma)

CPU times: user 1.67 s, sys: 3.18 s, total: 4.85 s
Wall time: 17min 29s


In [6]:
Data.head()

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,text,len,lem_text
date,,,,,,,,,
2017-11-09,132.309998,131.500000,132.820007,131.139999,33869650,0.809998,Компания рассчитывает на решение по газовому с...,419.0,компания рассчитывать на решение по газовый сп...
2017-11-08,132.000000,132.300003,133.940002,131.580002,39381960,-0.300003,"Как и предполагал “Ъ”, «Газпром», воспользова...",624.0,"как и предполагать “Ъ”, «газпром», воспользова..."
2017-11-01,126.400002,126.500000,126.889999,125.970001,18232550,-0.099998,Новая редакция американских санкций ставит по...,835.0,новый редакция американский санкция ставить по...
2017-10-30,125.959999,125.980003,126.930000,125.529999,19263340,-0.020004,"Как стало известно “Ъ”, известный на рынке ри...",436.0,"как становиться известно “Ъ”, известный на рын..."
2017-10-23,127.050003,126.800003,127.470001,126.370003,17308800,0.250000,"НОВАТЭК, который через пять лет собирается за...",1272.0,"новатэк, который через пять год собираться зап..."


In [7]:
D = Data.copy()

In [8]:
D['miller_count'] = D.lem_text.map(
    lambda text: text.count('миллер') if text else 0,
    na_action='ignore'
)
D['sever_count'] = D.lem_text.map(
    lambda text: text.count('северный поток') if text else 0,
    na_action='ignore'
)
D['turk_count'] = D.lem_text.map(
    lambda text: text.count('турецкий поток') if text else 0,
    na_action='ignore'
)

##### Ответьте на следующие вопросы:

    1.Есть ли корреляция между средней длинной текста за день и ценой закрытия?

In [9]:
CorrKoef = D.corr()
CorrKoef

,Open,ClosingPrice,DailyHigh,DailyLow,VolumePcs,delta,len,miller_count,sever_count,turk_count
Open,1.000000,0.995115,0.998069,0.997341,0.371362,0.081451,0.014575,0.006844,0.037291,-0.062884
ClosingPrice,0.995115,1.000000,0.997728,0.997670,0.376791,-0.017338,0.012376,0.008110,0.035715,-0.061107
DailyHigh,0.998069,0.997728,1.000000,0.996902,0.396443,0.035520,0.015082,0.007792,0.036612,-0.061706
DailyLow,0.997341,0.997670,0.996902,1.000000,0.346697,0.028728,0.010711,0.008442,0.037763,-0.059570
VolumePcs,0.371362,0.376791,0.396443,0.346697,1.000000,-0.042877,0.076835,0.007320,-0.011082,-0.100423
delta,0.081451,-0.017338,0.035520,0.028728,-0.042877,1.000000,0.022667,-0.012564,0.017115,-0.019967
len,0.014575,0.012376,0.015082,0.010711,0.076835,0.022667,1.000000,0.148300,0.081808,0.058477
miller_count,0.006844,0.008110,0.007792,0.008442,0.007320,-0.012564,0.148300,1.000000,0.164734,-0.016549
sever_count,0.037291,0.035715,0.036612,0.037763,-0.011082,0.017115,0.081808,0.164734,1.000000,0.075493
turk_count,-0.062884,-0.061107,-0.061706,-0.059570,-0.100423,-0.019967,0.058477,-0.016549,0.075493,1.000000


Судя по коэффициенту корреляции длинна текста не коррелирует ни с ценой закрытия, ни, даже с дельтой за день.

    2.Есть ли корреляция между количеством упоминаний Алексея Миллера и ценой закрытия? Учтите разные варианты написания имени.

    3.Упоминаний какого газопровода в статьях больше:
        "северный поток"
        "турецкий поток"?

    4.О каких санкциях пишут в статьях?

In [10]:
def save_texts(texts, file_name):
    with open(file_name, 'w') as f:
        for text in texts:
            f.write("{}\n".format(
                text.replace('\n', ' ').replace('\r', '').replace('\t', ' ')
            ))

save_texts(D.text, 'n_texts.txt')

In [ ]:
! cat ./n_texts.txt | docker run --rm -i inemo/syntaxnet_rus > news_texts.conll

Unable to find image 'inemo/syntaxnet_rus:latest' locally
latest: Pulling from inemo/syntaxnet_rus
7dcf5a444392: Pulling fs layer
759aa75f3cee: Pulling fs layer
3fa871dc8a2b: Pulling fs layer
224c42ae46e7: Pulling fs layer
db0093dc4b3e: Pulling fs layer
442196259211: Pulling fs layer
92cc81038f16: Pulling fs layer
0c381ee12dcf: Pulling fs layer
19ccddee82d3: Pulling fs layer
49a4ba5a54b7: Pulling fs layer
d639ba646fe3: Pulling fs layer
7cb3e0bdd6ac: Pulling fs layer
dbf2005010d3: Pulling fs layer
b36cc2eaa89d: Pulling fs layer
6662b081245f: Pulling fs layer
71b3b98bae00: Pulling fs layer
a852d3aa1a39: Pulling fs layer
afd97fa0ba3d: Pulling fs layer
c41a234e15ed: Pulling fs layer
991024d8d6c4: Pulling fs layer
0534e283426e: Pulling fs layer
7cb3e0bdd6ac: Waiting
dbf2005010d3: Waiting
b36cc2eaa89d: Waiting
224c42ae46e7: Waiting
6662b081245f: Waiting
db0093dc4b3e: Waiting
71b3b98bae00: Waiting
442196259211: Waiting
92cc81038f16: Waiting
0c381ee12dcf: Waiting
49a4ba5a54b7: Waiting
19ccddee

2017-12-30 19:38:31.076289: I syntaxnet/term_frequency_map.cc:101] Loaded 34 terms from ./syntaxnet/models/Russian-SynTagRus/label-map.
2017-12-30 19:38:31.076609: I syntaxnet/embedding_feature_extractor.cc:35] Features: input.capitalization input(1).capitalization input(2).capitalization input(3).capitalization input(-1).capitalization input(-2).capitalization input(-3).capitalization input(-4).capitalization; input.token.char-ngram input(1).token.char-ngram input(2).token.char-ngram input(3).token.char-ngram input(-1).token.char-ngram input(-2).token.char-ngram input(-3).token.char-ngram input(-4).token.char-ngram; input.digit input.hyphen input.token.punctuation-amount input.token.quote; input.token.prefix(length=2) input(1).token.prefix(length=2) input(2).token.prefix(length=2) input(3).token.prefix(length=2) input(-1).token.prefix(length=2) input(-2).token.prefix(length=2) input(-3).token.prefix(length=2) input(-4).token.prefix(length=2); input.token.prefix(length=3) input(1).toke

2017-12-30 19:38:37.121009: I syntaxnet/term_frequency_map.cc:101] Loaded 18749 terms from ./syntaxnet/models/Russian-SynTagRus/char-ngram-map.
2017-12-30 19:38:37.437652: I syntaxnet/term_frequency_map.cc:101] Loaded 103473 terms from ./syntaxnet/models/Russian-SynTagRus/word-map.
INFO:tensorflow:Restoring parameters from ./syntaxnet/models/Russian-SynTagRus/tagger-params
2017-12-30 19:38:37.624885: I syntaxnet/term_frequency_map.cc:101] Loaded 31 terms from ./syntaxnet/models/Russian-SynTagRus/tag-map.
2017-12-30 19:38:37.625814: I syntaxnet/term_frequency_map.cc:101] Loaded 34 terms from ./syntaxnet/models/Russian-SynTagRus/label-map.
2017-12-30 19:38:37.625852: I syntaxnet/embedding_feature_extractor.cc:35] Features: input.capitalization input(1).capitalization input(2).capitalization input(3).capitalization input(-1).capitalization input(-2).capitalization input(-3).capitalization input(-4).capitalization; input.token.char-ngram input(1).token.char-ngram input(2).token.char-ngram 

#### Часть 2. Классификационная

Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится цена закрытия. Для этого:

    бинаризуйте признак "цена закрытия": новый признак ClosingPrice_bin равен 1, если по сравнению со вчера цена не упала, и 0 – в обратном случаея;
    составьте бучающее и тестовое множество: данные до начала 2016 года используются для обучения, данные с 2016 года и позже – для тестирования.

Таким образом, в каждлый момент времени мы знаем:

    ClosingPrice_bin – бинарый целевой признак
    слова из статей, опубликованных в этот день – объясняющие признаки

В этой части задания вам нужно сделать baseline алгоритм и попытаться его улучшить в следующей части.

Используйте любой известный вам алгоритм классификации текстов для того, Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure для оценки качества классификации. Покажите, как $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации. Если у выбранного вами алгоритма есть гиперпараметры (например, $\alpha$ в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации.




Часть 3. Творческая

Придумайте и попытайтесь сделать еще что-нибудь, чтобы улучшить качество классификации. Направления развития:

    Морфологический признаки:
        использовать в качестве признаков только существительные или только именованные сущности;
    Модели скрытых тем:
        использовать в качестве признаков скрытые темы;
        использовать в качестве признаков динамические скрытые темы пример тут: (https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb)
    Синтаксические признаки:
        использовать SOV-тройки в качестве признаков
        кластеризовать SOV-тройки по эмбеддингам глаголов (обученные word2vec модели можно скачать отсюда: (http://rusvectores.org/ru/models/) и использовать только центроиды кластеров в качестве признаков
    что-нибудь еще

